In [1]:
import os
import time
import subprocess

def run_pose_compare_tool(video_path, label, tool_path, output_file="output_score.txt"):
    """
    运行 MiguPoseCompare 工具对视频进行评分，并返回评分结果。
    
    Args:
        video_path (str): 视频文件路径。
        label (int): 视频对应的动作 label。
        tool_path (str): MiguPoseCompare 工具所在路径。
        output_file (str): 工具生成的评分结果文件。
    
    Returns:
        float: 视频的评分结果。
    """
    # 确保当前目录是工具所在路径
    original_cwd = os.getcwd()
    os.chdir(tool_path)

    # 复制视频到工具路径
    video_name = os.path.basename(video_path)
    tool_video_path = os.path.join(tool_path, video_name)
    if not os.path.exists(tool_video_path):
        os.system(f'copy "{video_path}" "{tool_video_path}"')

    # 运行工具
    cmd = [
        os.path.join(tool_path, "MiguPoseCompare.exe"),
        "-i", video_name,
        "-label", str(label),
        "-o", output_file
    ]
    try:
        print(f"正在运行工具，对视频 {video_name} 进行评分...")
        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        process.wait()  # 等待工具运行完成
        
        # 等待结果文件生成
        output_path = os.path.join(tool_path, output_file)
        for _ in range(300):  # 最长等待 60 秒
            if os.path.exists(output_path):
                break
            time.sleep(1)
        else:
            raise FileNotFoundError(f"评分结果文件未生成: {output_path}")

        # 读取评分结果
        with open(output_path, "r") as f:
            score = float(f.read().strip())
            print(f"评分完成，得分: {score}")

        # 删除工具路径中的临时文件
        os.remove(tool_video_path)
        os.remove(output_path)

        # 重命名原始视频文件
        video_dir = os.path.dirname(video_path)
        new_video_name = f"{os.path.splitext(video_name)[0]}_{score:.6f}.mp4"
        new_video_path = os.path.join(video_dir, new_video_name)
        os.rename(video_path, new_video_path)
        print(f"原始视频已重命名为: {new_video_path}")

        # 返回评分结果
        return score
    except Exception as e:
        print(f"发生错误: {e}")
        return None
    finally:
        # 切回原始工作目录
        os.chdir(original_cwd)



In [ ]:

# 示例调用
video_path = r"C:\data\video\0-两手托天理三焦（八段锦）\reference_0.mp4"
label = 0
tool_path = r"C:\Users\Administrator\Desktop\code\MiguPoseCompare"
score = run_pose_compare_tool(video_path, label, tool_path)
if score is not None:
    print(f"视频评分为: {score}")


In [8]:
import os
import shutil

def copy_tool_folders(base_tool_path, num_folders):
    """
    复制工具文件夹到多个子文件夹。

    Args:
        base_tool_path (str): 原始工具文件夹路径。
        num_folders (int): 需要复制的子文件夹数量。

    Returns:
        list: 创建的子文件夹路径列表。
    """
    folder_paths = []
    for i in range(num_folders):
        folder_name = f"{os.path.basename(base_tool_path)}_{i}"
        folder_path = os.path.join(os.path.dirname(base_tool_path), folder_name)
        if not os.path.exists(folder_path):
            shutil.copytree(base_tool_path, folder_path)
        folder_paths.append(folder_path)
    return folder_paths

# 设置基础工具路径和目标数量
base_tool_path = r"C:\Users\Administrator\Desktop\code\MiguPoseCompare"
num_folders = 13

# 执行复制操作
created_folders = copy_tool_folders(base_tool_path, num_folders)

In [2]:
import os
from concurrent.futures import ThreadPoolExecutor

def process_videos_in_folder(folder_path, label, tool_path):
    """
    处理单个文件夹下的视频。
    Args:
        folder_path (str): 文件夹路径。
        label (int): 文件夹对应的动作 label。
        tool_path (str): 工具文件夹路径。
    """
    video_files = [
        os.path.join(folder_path, f)
        for f in os.listdir(folder_path)
        if f.endswith(".mp4")
    ]
    for video_file in video_files:
        print(f"开始处理视频: {video_file}")
        score = run_pose_compare_tool(video_file, label, tool_path)
        if score is not None:
            print(f"视频 {video_file} 评分完成，得分: {score}")
        else:
            print(f"视频 {video_file} 评分失败。")


def main():
    base_video_dir = r"C:\data\video"
    tool_base_dir = r"C:\Users\Administrator\Desktop\code"

    # 获取所有 0-13 开头的文件夹
    video_folders = [
        os.path.join(base_video_dir, d)
        for d in os.listdir(base_video_dir)
        if os.path.isdir(os.path.join(base_video_dir, d)) and d.split('-')[0].isdigit() and 0 <= int(d.split('-')[0]) <= 13
    ]

    # 启动多线程池，对每个文件夹并行处理
    with ThreadPoolExecutor(max_workers=13) as executor:
        for video_folder in video_folders:
            label = int(os.path.basename(video_folder).split('-')[0])  # 获取 label
            tool_path = os.path.join(tool_base_dir, f"MiguPoseCompare_{label}")  # 对应的工具路径

            # 提交任务：处理该文件夹
            executor.submit(process_videos_in_folder, video_folder, label, tool_path)


if __name__ == "__main__":
    main()


开始处理视频: C:\data\video\0-两手托天理三焦（八段锦）\reference_0.mp4
开始处理视频: C:\data\video\1-左右开弓似射雕（八段锦）\reference_1.mp4
开始处理视频: C:\data\video\2-调理脾胃单臂举（八段锦）\reference_2.mp4
开始处理视频: C:\data\video\3-五劳七伤往后瞧（八段锦）\reference_3.mp4
开始处理视频: C:\data\video\5-两手攀足固肾腰（八段锦）\reference_5.mp4
开始处理视频: C:\data\video\4-摇头摆尾去心火（八段锦）\reference_4.mp4
开始处理视频: C:\data\video\6-攒拳怒目增气力（八段锦）\reference_6.mp4
正在运行工具，对视频 reference_0.mp4 进行评分...
正在运行工具，对视频 reference_2.mp4 进行评分...
正在运行工具，对视频 reference_3.mp4 进行评分...
正在运行工具，对视频 reference_6.mp4 进行评分...
正在运行工具，对视频 reference_1.mp4 进行评分...
正在运行工具，对视频 reference_5.mp4 进行评分...
正在运行工具，对视频 reference_4.mp4 进行评分...
评分完成，得分: 0.714286
原始视频已重命名为: C:\data\video\6-攒拳怒目增气力（八段锦）\reference_6_0.714286.mp4
视频 C:\data\video\6-攒拳怒目增气力（八段锦）\reference_6.mp4 评分完成，得分: 0.714286
开始处理视频: C:\data\video\6-攒拳怒目增气力（八段锦）\standard_6.mp4
正在运行工具，对视频 standard_6.mp4 进行评分...
评分完成，得分: 1.0
原始视频已重命名为: C:\data\video\6-攒拳怒目增气力（八段锦）\standard_6_1.000000.mp4
视频 C:\data\video\6-攒拳怒目增气力（八段锦）\standard_6.mp4 评分完成，得分: 1.0
开始处理视频: C:\d

In [13]:
import os
import re

def check_video_processing_status(data_root):
    """
    检查0-13范围内的文件夹下的视频是否已经按照命名规则（_得分，6位小数）处理完成。
    如果视频文件名中没有得分（即没有包含 _得分），则输出未处理完成的文件路径。

    Args:
        data_root (str): 视频数据的根目录，包含多个文件夹。
    """
    # 遍历根目录中的文件夹
    for folder_name in os.listdir(data_root):
        # 提取文件夹前两个字符并从中提取数字
        folder_prefix = folder_name[:2]  # 获取前两个字符
        folder_digit = ''.join(filter(str.isdigit, folder_prefix))  # 提取其中的数字

        # 检查数字是否在 0-13 范围内
        if folder_digit.isdigit() and 0 <= int(folder_digit) <= 13:
            folder_path = os.path.join(data_root, folder_name)

            if not os.path.exists(folder_path):
                print(f"文件夹 {folder_path} 不存在！")
                continue

            # 遍历文件夹中的每个视频文件
            for video_file in os.listdir(folder_path):
                video_path = os.path.join(folder_path, video_file)

                # 只检查以 .mp4 结尾的文件
                if video_file.endswith(".mp4"):
                    # 检查文件名是否包含符合规则的得分格式 (_得分, 6位小数)
                    if not re.search(r'_\d\.\d{6}', video_file):
                        print(f"视频未处理完成: {video_file} (路径: {video_path})")


# 调用示例
data_root = r"C:\data\video"  # 设置根目录路径
check_video_processing_status(data_root)


In [8]:
import os
import time
import subprocess

def run_pose_compare_tool(video_path, label, tool_path, output_file="output_score.txt"):
    """
    运行 MiguPoseCompare 工具对视频进行评分，并返回评分结果。
    
    Args:
        video_path (str): 视频文件路径。
        label (int): 视频对应的动作 label。
        tool_path (str): MiguPoseCompare 工具所在路径。
        output_file (str): 工具生成的评分结果文件。
    
    Returns:
        float: 视频的评分结果。
    """
    # 确保当前目录是工具所在路径
    original_cwd = os.getcwd()
    os.chdir(tool_path)

    # 复制视频到工具路径
    video_name = os.path.basename(video_path)
    tool_video_path = os.path.join(tool_path, video_name)
    if not os.path.exists(tool_video_path):
        os.system(f'copy "{video_path}" "{tool_video_path}"')

    # 运行工具
    cmd = [
        os.path.join(tool_path, "MiguPoseCompare.exe"),
        "-i", video_name,
        "-label", str(label),
        "-o", output_file
    ]
    try:
        print(f"正在运行工具，对视频 {video_name} 进行评分...")
        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        process.wait()  # 等待工具运行完成
        
        # 等待结果文件生成
        output_path = os.path.join(tool_path, output_file)
        for _ in range(300):  # 最长等待 60 秒
            if os.path.exists(output_path):
                break
            time.sleep(1)
        else:
            raise FileNotFoundError(f"评分结果文件未生成: {output_path}")

        # 读取评分结果
        with open(output_path, "r") as f:
            score = float(f.read().strip())
            print(f"评分完成，得分: {score}")

        # 删除工具路径中的临时文件
        os.remove(tool_video_path)
        os.remove(output_path)

        # 重命名原始视频文件
        video_dir = os.path.dirname(video_path)
        new_video_name = f"{os.path.splitext(video_name)[0]}_{score:.6f}.mp4"
        new_video_path = os.path.join(video_dir, new_video_name)
        os.rename(video_path, new_video_path)
        print(f"原始视频已重命名为: {new_video_path}")

        # 返回评分结果
        return score
    except Exception as e:
        print(f"发生错误: {e}")
        return None
    finally:
        # 切回原始工作目录
        os.chdir(original_cwd)


def process_folders_7_to_13():
    base_video_dir = r"C:\data\video"
    tool_base_dir = r"C:\Users\Administrator\Desktop\code"

    # 获取以 7 到 13 开头的文件夹（没有 '-'）
    video_folders = [
        os.path.join(base_video_dir, d)
        for d in os.listdir(base_video_dir)
        if os.path.isdir(os.path.join(base_video_dir, d)) and d[0].isdigit() and 7 <= int(d[0]) <= 13
    ]

    # 确保按顺序处理 7-13 的文件夹
    video_folders.sort(key=lambda x: int(os.path.basename(x)[0]))  # 按文件夹名的数字部分排序

    # 启动多线程池，每个线程处理一个文件夹
    with ThreadPoolExecutor(max_workers=len(video_folders)) as executor:
        for video_folder in video_folders:
            label = int(os.path.basename(video_folder)[0])  # 获取文件夹名的第一位作为 label
            tool_path = os.path.join(tool_base_dir, f"MiguPoseCompare_{label}")  # 对应的工具路径

            # 提交任务：处理该文件夹
            executor.submit(process_videos_in_folder, video_folder, label, tool_path)


if __name__ == "__main__":
    process_folders_7_to_13()


开始处理视频: C:\data\video\7背后七颠百病消（八段锦）\42-动作7.mp4
开始处理视频: C:\data\video\8虎举（五禽戏）\reference_8.mp4
开始处理视频: C:\data\video\9虎扑（五禽戏）\reference_9.mp4
正在运行工具，对视频 42-动作7.mp4 进行评分...
正在运行工具，对视频 reference_9.mp4 进行评分...
正在运行工具，对视频 reference_8.mp4 进行评分...
评分完成，得分: 0.841111
原始视频已重命名为: C:\data\video\9虎扑（五禽戏）\reference_9_0.841111.mp4
视频 C:\data\video\9虎扑（五禽戏）\reference_9.mp4 评分完成，得分: 0.841111
开始处理视频: C:\data\video\9虎扑（五禽戏）\standard_9.mp4
正在运行工具，对视频 standard_9.mp4 进行评分...
评分完成，得分: 1.0
原始视频已重命名为: C:\data\video\9虎扑（五禽戏）\standard_9_1.000000.mp4
视频 C:\data\video\9虎扑（五禽戏）\standard_9.mp4 评分完成，得分: 1.0
开始处理视频: C:\data\video\9虎扑（五禽戏）\动作9-1-12.mp4
正在运行工具，对视频 动作9-1-12.mp4 进行评分...
评分完成，得分: 0.735556
原始视频已重命名为: C:\data\video\9虎扑（五禽戏）\动作9-1-12_0.735556.mp4
视频 C:\data\video\9虎扑（五禽戏）\动作9-1-12.mp4 评分完成，得分: 0.735556
开始处理视频: C:\data\video\9虎扑（五禽戏）\动作9-1-17.mp4
正在运行工具，对视频 动作9-1-17.mp4 进行评分...
评分完成，得分: 0.674444
原始视频已重命名为: C:\data\video\9虎扑（五禽戏）\动作9-1-17_0.674444.mp4
视频 C:\data\video\9虎扑（五禽戏）\动作9-1-17.mp4 评分完成，得分: 0.674444
开始处

In [10]:
def process_folders_10_to_13():
    base_video_dir = r"C:\data\video"
    tool_base_dir = r"C:\Users\Administrator\Desktop\code"

    # 获取以 10 到 13 开头的文件夹（没有 '-'）
    video_folders = [
        os.path.join(base_video_dir, d)
        for d in os.listdir(base_video_dir)
        if os.path.isdir(os.path.join(base_video_dir, d)) and d[:2].isdigit() and 10 <= int(d[:2]) <= 13
    ]

    # 确保按顺序处理 10-13 的文件夹
    video_folders.sort(key=lambda x: int(os.path.basename(x)[:2]))  # 按文件夹名的前两位数字排序

    # 启动多线程池，每个线程处理一个文件夹
    with ThreadPoolExecutor(max_workers=len(video_folders)) as executor:
        for video_folder in video_folders:
            label = int(os.path.basename(video_folder)[:2])  # 获取文件夹名的前两位作为 label
            tool_path = os.path.join(tool_base_dir, f"MiguPoseCompare_{label}")  # 对应的工具路径

            # 提交任务：处理该文件夹
            executor.submit(process_videos_in_folder, video_folder, label, tool_path)


if __name__ == "__main__":
    process_folders_10_to_13()

开始处理视频: C:\data\video\10鹿抵（五禽戏）\reference_10.mp4
开始处理视频: C:\data\video\11鹿奔（五禽戏）\41-动作11.mp4
开始处理视频: C:\data\video\13鸟飞（五禽戏）\reference_13.mp4
开始处理视频: C:\data\video\12鸟伸（五禽戏）\44-动作12.mp4
正在运行工具，对视频 44-动作12.mp4 进行评分...
正在运行工具，对视频 41-动作11.mp4 进行评分...
正在运行工具，对视频 reference_13.mp4 进行评分...
正在运行工具，对视频 reference_10.mp4 进行评分...
评分完成，得分: 0.815714
原始视频已重命名为: C:\data\video\12鸟伸（五禽戏）\44-动作12_0.815714.mp4
视频 C:\data\video\12鸟伸（五禽戏）\44-动作12.mp4 评分完成，得分: 0.815714
开始处理视频: C:\data\video\12鸟伸（五禽戏）\reference_12.mp4
正在运行工具，对视频 reference_12.mp4 进行评分...
评分完成，得分: 0.855714
原始视频已重命名为: C:\data\video\12鸟伸（五禽戏）\reference_12_0.855714.mp4
视频 C:\data\video\12鸟伸（五禽戏）\reference_12.mp4 评分完成，得分: 0.855714
开始处理视频: C:\data\video\12鸟伸（五禽戏）\standard_12.mp4
正在运行工具，对视频 standard_12.mp4 进行评分...
评分完成，得分: 1.0
原始视频已重命名为: C:\data\video\12鸟伸（五禽戏）\standard_12_1.000000.mp4
视频 C:\data\video\12鸟伸（五禽戏）\standard_12.mp4 评分完成，得分: 1.0
开始处理视频: C:\data\video\12鸟伸（五禽戏）\动作12-1-21.mp4
正在运行工具，对视频 动作12-1-21.mp4 进行评分...
评分完成，得分: 0.801429
原始视频已重命名为: C:\

In [12]:
def check_video_processing_status(data_root):
    """
    检查视频文件是否未处理并返回未处理的视频集合。
    """
    unprocessed_videos = []

    for folder_name in os.listdir(data_root):
        # 提取文件夹前两个字符并从中提取数字
        folder_prefix = folder_name[:2]
        folder_digit = ''.join(filter(str.isdigit, folder_prefix))  # 提取数字部分

        # 检查数字是否在 0-13 范围内
        if folder_digit.isdigit() and 0 <= int(folder_digit) <= 13:
            folder_path = os.path.join(data_root, folder_name)

            if not os.path.exists(folder_path):
                print(f"文件夹 {folder_path} 不存在！")
                continue

            # 遍历文件夹中的每个视频文件
            for video_file in os.listdir(folder_path):
                video_path = os.path.join(folder_path, video_file)

                # 检查 .mp4 文件名是否包含得分后缀
                if video_file.endswith(".mp4") and not re.search(r'_\d\.\d{6}', video_file):
                    label = int(folder_digit)  # 使用文件夹数字作为动作标签
                    unprocessed_videos.append((video_path, label))  # 收集未处理视频信息

    return unprocessed_videos


def process_unprocessed_videos(unprocessed_videos, tool_base_dir):
    """
    依次处理未处理的视频集合。
    """
    for video_path, label in unprocessed_videos:
        # 构建工具路径
        tool_path = os.path.join(tool_base_dir, f"MiguPoseCompare_{label}")

        # 调用工具进行处理
        score = run_pose_compare_tool(video_path, label, tool_path)
        if score is not None:
            print(f"视频 {video_path} 评分完成，得分: {score}")
        else:
            print(f"视频 {video_path} 评分失败。")


if __name__ == "__main__":
    # 设置根目录和工具根路径
    data_root = r"C:\data\video"
    tool_base_dir = r"C:\Users\Administrator\Desktop\code"

    # 检查未处理的视频
    unprocessed_videos = check_video_processing_status(data_root)

    # 打印未处理的视频信息
    print(f"发现 {len(unprocessed_videos)} 个未处理的视频：")
    for video_path, label in unprocessed_videos:
        print(f"未处理视频: {video_path}, 动作标签: {label}")

    # 处理未处理的视频
    process_unprocessed_videos(unprocessed_videos, tool_base_dir)

发现 23 个未处理的视频：
未处理视频: C:\data\video\0-两手托天理三焦（八段锦）\reference_0.mp4, 动作标签: 0
未处理视频: C:\data\video\1-左右开弓似射雕（八段锦）\reference_1.mp4, 动作标签: 1
未处理视频: C:\data\video\1-左右开弓似射雕（八段锦）\standard_1.mp4, 动作标签: 1
未处理视频: C:\data\video\10鹿抵（五禽戏）\reference_10.mp4, 动作标签: 10
未处理视频: C:\data\video\11鹿奔（五禽戏）\41-动作11.mp4, 动作标签: 11
未处理视频: C:\data\video\11鹿奔（五禽戏）\reference_11.mp4, 动作标签: 11
未处理视频: C:\data\video\11鹿奔（五禽戏）\动作11-3-5.mp4, 动作标签: 11
未处理视频: C:\data\video\11鹿奔（五禽戏）\动作11-4-23.mp4, 动作标签: 11
未处理视频: C:\data\video\12鸟伸（五禽戏）\动作12-5-93.mp4, 动作标签: 12
未处理视频: C:\data\video\13鸟飞（五禽戏）\reference_13.mp4, 动作标签: 13
未处理视频: C:\data\video\13鸟飞（五禽戏）\standard_13.mp4, 动作标签: 13
未处理视频: C:\data\video\13鸟飞（五禽戏）\动作13-1-11.mp4, 动作标签: 13
未处理视频: C:\data\video\2-调理脾胃单臂举（八段锦）\reference_2.mp4, 动作标签: 2
未处理视频: C:\data\video\3-五劳七伤往后瞧（八段锦）\reference_3.mp4, 动作标签: 3
未处理视频: C:\data\video\3-五劳七伤往后瞧（八段锦）\standard_3.mp4, 动作标签: 3
未处理视频: C:\data\video\3-五劳七伤往后瞧（八段锦）\动作3-1-33.mp4, 动作标签: 3
未处理视频: C:\data\video\3-五劳七伤往后瞧（八段锦）\动作3-3-33.mp4, 动作标签: 3
未处